In [1]:
library(tidyverse) #Reading data
library(lubridate) #Reading and manipulating dates and times
library(readr) #Reading the CSV data file
library(DBI)
library(RMariaDB)
library(RSQLite)

Warning message:
"package 'tidyverse' was built under R version 4.3.3"


Warning message:
"package 'ggplot2' was built under R version 4.3.3"
Warning message:
"package 'tibble' was built under R version 4.3.3"
Warning message:
"package 'tidyr' was built under R version 4.3.3"
Warning message:
"package 'readr' was built under R version 4.3.3"
Warning message:
"package 'dplyr' was built under R version 4.3.3"
Warning message:
"package 'forcats' was built under R version 4.3.3"
Warning message:
"package 'lubridate' was built under R version 4.3.3"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become error

In [2]:
raw_stroke_data = read.csv("healthcare_dataset_stroke_data.csv")
stroke_data = data.frame(raw_stroke_data)

In [3]:
glimpse(stroke_data)

Rows: 5,110
Columns: 12
$ id                <int> 9046, 51676, 31112, 60182, 1665, 56669, 53882, 10434…
$ gender            <chr> "Male", "Female", "Male", "Female", "Female", "Male"…
$ age               <dbl> 67, 61, 80, 49, 79, 81, 74, 69, 59, 78, 81, 61, 54, …
$ hypertension      <int> 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1…
$ heart_disease     <int> 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0…
$ ever_married      <chr> "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "Yes", "No…
$ work_type         <chr> "Private", "Self-employed", "Private", "Private", "S…
$ Residence_type    <chr> "Urban", "Rural", "Rural", "Urban", "Rural", "Urban"…
$ avg_glucose_level <dbl> 228.69, 202.21, 105.92, 171.23, 174.12, 186.21, 70.0…
$ bmi               <chr> "36.6", "N/A", "32.5", "34.4", "24", "29", "27.4", "…
$ smoking_status    <chr> "formerly smoked", "never smoked", "never smoked", "…
$ stroke            <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…


In [4]:
stroke_data = rename(stroke_data, residence_type = Residence_type)
colnames(stroke_data)

[1] "id"                "gender"            "age"              
 [4] "hypertension"      "heart_disease"     "ever_married"     
 [7] "work_type"         "residence_type"    "avg_glucose_level"
[10] "bmi"               "smoking_status"    "stroke"

In [5]:
missing_values = colSums(is.na(stroke_data))
print(missing_values)

               id            gender               age      hypertension 
                0                 0                 0                 0 
    heart_disease      ever_married         work_type    residence_type 
                0                 0                 0                 0 
avg_glucose_level               bmi    smoking_status            stroke 
                0                 0                 0                 0 


In [6]:
print("original names")
print(sapply(stroke_data, class))

[1] "original names"
               id            gender               age      hypertension 
        "integer"       "character"         "numeric"         "integer" 
    heart_disease      ever_married         work_type    residence_type 
        "integer"       "character"       "character"       "character" 
avg_glucose_level               bmi    smoking_status            stroke 
        "numeric"       "character"       "character"         "integer" 


In [7]:
head(stroke_data, n=5)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
,<int>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>
1,9046,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,51676,Female,61,0,0,Yes,Self-employed,Rural,202.21,N/A,never smoked,1
3,31112,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
4,60182,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
5,1665,Female,79,1,0,Yes,Self-employed,Rural,174.12,24,never smoked,1


In [8]:
# BMI is the only column with N/A so I will check if I can just drop those rows without signifcant data loss
# I'll make not of that in my analysis
stroke_data_na_removed = filter(stroke_data, !grepl("N/A", bmi))

print("With the rows that have N/A")
with_rows = nrow(stroke_data)
print(with_rows)

print("Without the rows with N/A")
without_rows = nrow(stroke_data_na_removed)
print(without_rows)

print("Difference")
print(with_rows - without_rows)

[1] "With the rows that have N/A"
[1] 5110
[1] "Without the rows with N/A"
[1] 4909
[1] "Difference"
[1] 201


In [9]:
stroke_data = select(stroke_data, -id)
stroke_data = mutate(stroke_data, index = row_number())
stroke_data = rename(stroke_data, person_id = index)
colnames(stroke_data)

[1] "gender"            "age"               "hypertension"     
 [4] "heart_disease"     "ever_married"      "work_type"        
 [7] "residence_type"    "avg_glucose_level" "bmi"              
[10] "smoking_status"    "stroke"            "person_id"

In [10]:
cleaned_datset = data.frame(stroke_data)
db = dbConnect(dbDriver("SQLite"), dbname="stroke_data.db")

In [11]:
first_table = "CREATE TABLE IF NOT EXISTS person(
                person_id INTEGER PRIMARY KEY,
                gender TEXT,
                age INTEGER,
                ever_married TEXT,
                work_type TEXT,
                residence_type TEXT
);"

dbExecute(db, first_table)
dbListTables(db)

[1] 0

[1] "person"

In [12]:
table_name="person"
query_table_1 = paste0("PRAGMA table_info(",table_name,");")
db_columns = dbGetQuery(db,query_table_1)$name
common_columns_first = intersect(colnames(cleaned_datset), db_columns)
common_columns_first

[1] "gender"         "age"            "ever_married"   "work_type"     
[5] "residence_type" "person_id"

In [13]:
person_subset = cleaned_datset[,common_columns_first]
dbWriteTable(db,"person",person_subset, append = TRUE, row.names = FALSE)
dbGetQuery(db,"SELECT * 
                FROM person
                LIMIT 5")

person_id,gender,age,ever_married,work_type,residence_type
<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,Male,67,Yes,Private,Urban
2,Female,61,Yes,Self-employed,Rural
3,Male,80,Yes,Private,Rural
4,Female,49,Yes,Private,Urban
5,Female,79,Yes,Self-employed,Rural


In [14]:
second_table = "CREATE TABLE IF NOT EXISTS medical_history(
                person_id INTEGER,
                hypertension INTEGER,
                heart_disease INTEGER,
                avg_glucose_level REAL,
                smoking_status TEXT,
                stroke INTEGER,
                bmi REAL,
                FOREIGN KEY (person_id) REFERENCES person(person_id)
);"

dbExecute(db, second_table)
dbListTables(db)

[1] 0

[1] "medical_history" "person"

In [15]:
table_name = "medical_history"
query_table_2 = paste0("PRAGMA table_info(", table_name,");")
db_columns = dbGetQuery(db, query_table_2)$name
common_columns_second = intersect(colnames(cleaned_datset),db_columns)
common_columns_second

[1] "hypertension"      "heart_disease"     "avg_glucose_level"
[4] "bmi"               "smoking_status"    "stroke"           
[7] "person_id"

In [16]:
medical_subset = cleaned_datset[, common_columns_second]
dbWriteTable(db, "medical_history", medical_subset, append=TRUE,row.names =FALSE)
dbGetQuery(db,"SELECT * 
                FROM medical_history
                LIMIT 5")

Warning message:
"Column `bmi`: mixed type, first seen values of type real, coercing other values of type string"


person_id,hypertension,heart_disease,avg_glucose_level,smoking_status,stroke,bmi
<int>,<int>,<int>,<dbl>,<chr>,<int>,<dbl>
1,0,1,228.69,formerly smoked,1,36.6
2,0,0,202.21,never smoked,1,0.0
3,0,1,105.92,never smoked,1,32.5
4,0,0,171.23,smokes,1,34.4
5,1,0,174.12,never smoked,1,24.0


In [18]:
totals = dbGetQuery(db,"SELECT 
                            COUNT(CASE WHEN m.stroke = 1 THEN 1 END) as  Had_a_stroke,
                            COUNT(CASE WHEN m.stroke = 0 THEN 1 END) as Havent_had_a_stroke,
                            COUNT(CASE WHEN m.stroke = 1 AND p.gender = 'Male' THEN 1 END) as Man_that_have_had_a_stroke,
                            COUNT(CASE WHEN m.stroke = 1 AND p.gender = 'Female' THEN 1 END) as Women_that_have_had_a_stroke,
                            COUNT(CASE WHEN m.stroke = 1 AND p.gender = 'Female' AND p.ever_married = 'Yes' THEN 1 END) as Women_married_had_stroke,
                            COUNT(CASE WHEN m.stroke = 1 AND p.gender = 'Male' AND p.ever_married = 'Yes' THEN 1 END) as Man_married_had_stroke
                        FROM
                            person p
                        LEFT JOIN medical_history m on p.person_id = m.person_id
")

totals
write.csv(totals,'total.csv')

Had_a_stroke,Havent_had_a_stroke,Man_that_have_had_a_stroke,Women_that_have_had_a_stroke,Women_married_had_stroke,Man_married_had_stroke
<int>,<int>,<int>,<int>,<int>,<int>
249,4861,108,141,120,100


In [19]:
BMI = dbGetQuery(db,"SELECT 
                            p.age,
                            p.gender,
                            m.bmi,
                            CASE 
                                WHEN m.stroke = 1 THEN 1 
                                ELSE 0 
                                END as stroke
                        FROM 
                            person p 
                        LEFT JOIN 
                            medical_history m on p.person_id = m.person_id
")
head(BMI)
write.csv(BMI,'bmi.csv')

Warning message:
"Column `bmi`: mixed type, first seen values of type real, coercing other values of type string"


,age,gender,bmi,stroke
,<dbl>,<chr>,<dbl>,<int>
1,67,Male,36.6,1
2,61,Female,0.0,1
3,80,Male,32.5,1
4,49,Female,34.4,1
5,79,Female,24.0,1
6,81,Male,29.0,1


In [20]:
Residence = dbGetQuery(db,"SELECT 
                                p.age,
                                p.gender,
                                p.residence_type,
                                CASE 
                                    WHEN m.stroke = 1 THEN 1 
                                    ELSE 0 
                                    END as stroke
                            FROM 
                                person p
                            LEFT JOIN 
                                medical_history m on p.person_id = m.person_id
")
head(Residence)
write.csv(Residence,"residence.csv")

,age,gender,residence_type,stroke
,<dbl>,<chr>,<chr>,<int>
1,67,Male,Urban,1
2,61,Female,Rural,1
3,80,Male,Rural,1
4,49,Female,Urban,1
5,79,Female,Rural,1
6,81,Male,Urban,1


In [21]:
Glucose = dbGetQuery(db,"SELECT
                                    p.gender,
                                    p.age,
                                    m.avg_glucose_level,
                                    CASE 
                                        WHEN m.stroke = 1 THEN 1 
                                        ELSE 0 
                                        END as stroke
                                FROM 
                                    person p
                                LEFT JOIN
                                    medical_history m on p.person_id = m.person_id
")

head(Glucose)
write.csv(Glucose,"glucose.csv")

,gender,age,avg_glucose_level,stroke
,<chr>,<dbl>,<dbl>,<int>
1,Male,67,228.69,1
2,Female,61,202.21,1
3,Male,80,105.92,1
4,Female,49,171.23,1
5,Female,79,174.12,1
6,Male,81,186.21,1
